In [3]:
#Import Required Libraries
import pandas as pd
import nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahmo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahmo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = pd.read_csv('C:\\Users\\Mahmo\\Coding\\NLP Project\\IMDB Dataset.csv')

df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [7]:
df = df.drop_duplicates()


In [9]:
#Converting to lowercase
df = df.map(lambda x: x.lower() if isinstance(x, str) else x)
#Removing remove non-word and non-whitespace characters
df = df.replace(to_replace=r'[^\w\s]', value='', regex=True)
# Convert 'text' column to string data type
df['review'] = df['review'].astype(str)  
  # Tokenization
df['tokens'] = df['review'].apply(nltk.word_tokenize)

In [10]:
df

,review,sentiment,tokens
0,one of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione..."
1,a wonderful little production br br the filmin...,positive,"[a, wonderful, little, production, br, br, the..."
2,i thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,..."
3,basically theres a family where a little boy j...,negative,"[basically, theres, a, family, where, a, littl..."
4,petter matteis love in the time of money is a ...,positive,"[petter, matteis, love, in, the, time, of, mon..."
...,...,...,...
49995,i thought this movie did a down right good job...,positive,"[i, thought, this, movie, did, a, down, right,..."
49996,bad plot bad dialogue bad acting idiotic direc...,negative,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,i am a catholic taught in parochial elementary...,negative,"[i, am, a, catholic, taught, in, parochial, el..."
49998,im going to have to disagree with the previous...,negative,"[im, going, to, have, to, disagree, with, the,..."


In [11]:
import re

# Define a regex pattern to match URLs
url_pattern = re.compile(r'https?://\S+')

# Define a function to remove URLs from text
def remove_urls(text):
    return url_pattern.sub('', text)

# Apply the function to the 'text' column and create a new column 'clean_text'
df['review'] = df['review'].apply(remove_urls)

In [17]:
df

,review,sentiment,tokens
0,one of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione..."
1,a wonderful little production br br the filmin...,positive,"[a, wonderful, little, production, br, br, the..."
2,i thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,..."
3,basically theres a family where a little boy j...,negative,"[basically, theres, a, family, where, a, littl..."
4,petter matteis love in the time of money is a ...,positive,"[petter, matteis, love, in, the, time, of, mon..."
...,...,...,...
49995,i thought this movie did a down right good job...,positive,"[i, thought, this, movie, did, a, down, right,..."
49996,bad plot bad dialogue bad acting idiotic direc...,negative,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,i am a catholic taught in parochial elementary...,negative,"[i, am, a, catholic, taught, in, parochial, el..."
49998,im going to have to disagree with the previous...,negative,"[im, going, to, have, to, disagree, with, the,..."


In [19]:
# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])

In [87]:
df

,review,sentiment,tokens
0,one of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, 1, oz, e..."
1,a wonderful little production br br the filmin...,positive,"[wonderful, little, production, br, br, filmin..."
2,i thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su..."
3,basically theres a family where a little boy j...,negative,"[basically, theres, family, little, boy, jake,..."
4,petter matteis love in the time of money is a ...,positive,"[petter, matteis, love, time, money, visually,..."
...,...,...,...
49995,i thought this movie did a down right good job...,positive,"[thought, movie, right, good, job, wasnt, crea..."
49996,bad plot bad dialogue bad acting idiotic direc...,negative,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,i am a catholic taught in parochial elementary...,negative,"[catholic, taught, parochial, elementary, scho..."
49998,im going to have to disagree with the previous...,negative,"[im, going, disagree, previous, comment, side,..."


In [27]:
# Apply stemming and lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

df['tokens_stemmed'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])
df['tokens_lemmatized'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [28]:
df['processed_review'] = df['tokens_lemmatized'].apply(lambda x: ' '.join(x))


In [97]:
df

,review,sentiment,tokens,tokens_stemmed,tokens_lemmatized,processed_review
0,one of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, 1, oz, e...","[one, review, mention, watch, 1, oz, episod, y...","[one, reviewer, mentioned, watching, 1, oz, ep...",one reviewer mentioned watching 1 oz episode y...
1,a wonderful little production br br the filmin...,positive,"[wonderful, little, production, br, br, filmin...","[wonder, littl, product, br, br, film, techniq...","[wonderful, little, production, br, br, filmin...",wonderful little production br br filming tech...
2,i thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su...","[thought, wonder, way, spend, time, hot, summe...","[thought, wonderful, way, spend, time, hot, su...",thought wonderful way spend time hot summer we...
3,basically theres a family where a little boy j...,negative,"[basically, theres, family, little, boy, jake,...","[basic, there, famili, littl, boy, jake, think...","[basically, there, family, little, boy, jake, ...",basically there family little boy jake think t...
4,petter matteis love in the time of money is a ...,positive,"[petter, matteis, love, time, money, visually,...","[petter, mattei, love, time, money, visual, st...","[petter, matteis, love, time, money, visually,...",petter matteis love time money visually stunni...
...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,"[thought, movie, right, good, job, wasnt, crea...","[thought, movi, right, good, job, wasnt, creat...","[thought, movie, right, good, job, wasnt, crea...",thought movie right good job wasnt creative or...
49996,bad plot bad dialogue bad acting idiotic direc...,negative,"[bad, plot, bad, dialogue, bad, acting, idioti...","[bad, plot, bad, dialogu, bad, act, idiot, dir...","[bad, plot, bad, dialogue, bad, acting, idioti...",bad plot bad dialogue bad acting idiotic direc...
49997,i am a catholic taught in parochial elementary...,negative,"[catholic, taught, parochial, elementary, scho...","[cathol, taught, parochi, elementari, school, ...","[catholic, taught, parochial, elementary, scho...",catholic taught parochial elementary school nu...
49998,im going to have to disagree with the previous...,negative,"[im, going, disagree, previous, comment, side,...","[im, go, disagre, previou, comment, side, malt...","[im, going, disagree, previous, comment, side,...",im going disagree previous comment side maltin...


In [99]:

# Feature Extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_review'])



In [100]:
# Convert 'sentiment' column to binary (assuming sentiment is 'positive' or 'negative')
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)



In [103]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)



In [105]:
# Model Training
model = SVC(kernel='linear', C=1)  # You can adjust parameters based on the performance
model.fit(X_train, y_train)


SVC(C=1, kernel='linear')

In [107]:

# Model Prediction
y_pred = model.predict(X_test)



In [109]:
# Model Evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4939
           1       0.89      0.91      0.90      4978

    accuracy                           0.90      9917
   macro avg       0.90      0.90      0.90      9917
weighted avg       0.90      0.90      0.90      9917

Accuracy Score: 0.8956337602097408


In [1]:
import pickle

# Saving the model as a pickle file
with open('svm_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Saving the TfidfVectorizer as a pickle file
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print("Model and vectorizer saved as pickle files.")

NameError: name 'model' is not defined